# 3. Thống kê suy diễn

In [191]:
import numpy as np 
import pandas as pd # Đọc file CSV, Làm việc với dataframe
import matplotlib.pyplot as plt # Sử dụng cho mục đích trực quan hóa dữ liệu
import seaborn as sns # Trực quan hóa dữ liệu
from scipy import stats as st # Kiểm định thống kê
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [192]:
# load data
data = pd.read_csv('Lnfix.csv', encoding= 'utf-8')
data.iloc[1949]

artist                               abec
author                      Reki Kawahara
gernes                  Adventure,Fantasy
heart                                 172
name      Sword Art Online- Hopeful Chant
star                                  4.7
view                                31708
words                               13081
Name: 1949, dtype: object

In [193]:
def results(p, vals=['mean1', 'mean2'], alpha=0.05):
    cols=['score', 'p_value', 'KetLuan']
    if p['p_value'] < alpha:
        p['KetLuan'] = f"Chấp nhận H1 với mức ý nghĩa {alpha}"
    if p['p_value'] >= alpha:
        p['KetLuan'] = f"Chấp nhận H0 với mức ý nghĩa {alpha}"
    df = pd.DataFrame(p, index=[''])
    if vals: cols = vals + cols
    return df[cols]


def results_1(p, alpha=0.05):
    cols=['score', 'p_value', 'dof', 'KetLuan']
    if p['p_value'] < alpha:
        p['KetLuan'] = f"Chấp nhận H1 với mức ý nghĩa {alpha}"
    if p['p_value'] >= alpha:
        p['KetLuan'] = f"Chấp nhận H0 với mức ý nghĩa {alpha}"
    df = pd.DataFrame(p, index=[''])
    return df[cols]

In [194]:
# Xem thông tin dữ liệu
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   artist  1949 non-null   object 
 1   author  1947 non-null   object 
 2   gernes  1950 non-null   object 
 3   heart   1950 non-null   int64  
 4   name    1950 non-null   object 
 5   star    1950 non-null   float64
 6   view    1950 non-null   int64  
 7   words   1950 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 122.0+ KB


In [195]:
df_1 = data.copy()
list = []
for i in df_1['artist']:
  if i == 'None':
    list.append('None')
  elif i == 'Không có':
    list.append('None')
  else:
    list.append('Has')
df_1.insert(8, 'Type_artist', list)

In [196]:
df_1.head(5)

,artist,author,gernes,heart,name,star,view,words,Type_artist
0,None,Phantom Of The Moonlit Night,"Action,Adventure,Comedy,Fantasy,Harem,Seinen,S...",132,Running Away From The Hero! (Remake),5.0,4071,6986,None
1,Shizuma Yoshinori (しずまよしのり),Shuu (秋),"Action,Adventure,Comedy,Fantasy,Harem,Romance,...",6651,Maou Gakuin no Futekigousha ~Shijou Saikyou no...,4.8,5978437,636903,Has
2,Fuumi,Itsuki Mizuho,"Adventure,Comedy,Fantasy,Slice of Life,Superna...",130,The Novice Alchemist's Store,5.0,7644,30539,Has
3,Macchi | 마끼,이윤아,"Drama,Fantasy,Josei,Korean Novel,Mature,Romance",51,Lời Cầu Hôn Của Bạo Chúa,4.6,10419,43480,Has
4,sime,cadet,"Action,Drama,Fantasy,Romance,School Life,Seine...",1832,"Ori of the Dragon Chain - ""Heart"" in the mind",4.8,358243,85317,Has


### 1. Dùng kiểm định T-test với sample ngẫu nhiên (n=30) để so sánh star theo artist
**H0**: $\mu_1 = \mu_2$

**H1:** $\mu_1 \ne \mu_2$

Lấy mẫu mỗi nhóm (size=30):

In [197]:
gr1 = df_1[df_1['Type_artist'] == 'Has'].sample(30)['star']
gr1.name = 'Has artist'
gr2 = df_1[df_1['Type_artist'] == 'None'].sample(30)['star']
gr2.name = 'No artist'
pd.concat([gr1, gr2], axis = 1).sample(5)


,Has artist,No artist
1863,5.0,NaN
1460,4.9,NaN
399,2.4,NaN
78,5.0,NaN
1474,4.9,NaN


In [198]:
# Với mức ý nghĩa 5% thì đánh giá của LN/WN có hình minh hoạ có khác nhau hay không

p = {}
p['mean1'] = np.mean(gr1)
p['mean2'] = np.mean(gr2)
p['score'], p['p_value'] = st.ttest_ind(gr1, gr2,equal_var=False)
print('Kiểm định t test 2 sample')
results(p)

Kiểm định t test 2 sample


,mean1,mean2,score,p_value,KetLuan
,4.736667,4.646667,0.777938,0.439858,Chấp nhận H0 với mức ý nghĩa 0.05


### 2 Dùng kiểm định Chi2 để nghiên cứu ảnh hưởng của View (số lượt xem) đến Words (số chữ)
#### Phát biểu giả thiết

$H_0: \mu_0 = \mu_1$. View không ảnh hưởng đến Words

$H_1: \mu_0 \neq \mu_1$. View ảnh hưởng đến Words

##### Từ cột view tạo cột Type_View

In [199]:
df_2 = data.copy()

In [200]:
Type_view = []
for i in df_2['view']:
  if i <= 20000:
    Type_view.append('Low')
  elif 20000 < i <= 70000:
    Type_view.append('Medium')
  elif 70000 < i <= 150000:
    Type_view.append('High')
  elif 150000 < i <= 300000:
    Type_view.append('Super_High')
  else:
    Type_view.append('Mega_High') 
df_2['Type_view'] = pd.Series(Type_view)

##### Từ cột word tạo cột Type_words

In [201]:
Type_words = []
for i in df_2['words']:
  if i <= 15000:
    Type_words.append('Low')
  elif 15000 < i <= 60000:
    Type_words.append('Medium')
  elif 60000 < i <= 150000:
    Type_words.append('High')
  elif 150000 < i <= 300000:
    Type_words.append('Super_High')
  else:
    Type_words.append('Mega_High')
df_2['Type_words'] = pd.Series(Type_words)
df_2.sample(5)

,artist,author,gernes,heart,name,star,view,words,Type_view,Type_words
624,None,Kamono Udon,"Action,Adventure,Fantasy,Web Novel",270,"Hazure Skill ‘Mapping’ wo Te ni Shita Ore wa, ...",4.8,33531,13711,Medium,Low
419,らむ屋,Nattou Gohan,"Adventure,Comedy,Ecchi,Gender Bender,Fantasy",706,Kuishinbo Elf,4.9,224657,142826,Super_High,High
530,Sikorsky,Taketsuki Jou,"Action,Adventure,Ecchi,Fantasy,Harem,Shounen,S...",301,Campione!,4.8,79641,152817,High,Super_High
1660,Momomochi,Hazama Takashi,"Adapted to Manga,Comedy,Drama,Psychological,Ro...",93,Asahina Wakaba to Marumaru na Kareshi,5.0,4088,5623,Low,Low
1938,Yuuji Nimura,Taketsuki Jou,"Action,Fantasy,Romance",121,Meiyaku no Leviathan,3.7,9860,1059,Low,Low


In [202]:
Type_view_words = df_2[['Type_view', 'Type_words']].copy()
Type_view_words.sample(5)

,Type_view,Type_words
192,Medium,Medium
399,Low,Low
173,Medium,Medium
1777,High,High
1282,Low,Low


In [203]:
tanso_df = pd.crosstab(index=Type_view_words['Type_view'], columns=Type_view_words['Type_words'])
tanso_df

Type_words,High,Low,Medium,Mega_High,Super_High
Type_view,,,,,
High,76,12,133,12,27
Low,8,498,126,0,2
Medium,112,249,328,4,14
Mega_High,49,0,6,76,59
Super_High,68,1,30,17,43


In [204]:
score, p_value, dof, expected = st.chi2_contingency(tanso_df)

In [205]:
p ={}
p['score'] = score
p['p_value'] = p_value
p['dof'] = dof
results_1(p)

,score,p_value,dof,KetLuan
,1606.5026,0.0,16,Chấp nhận H1 với mức ý nghĩa 0.05


#### 3. Dùng kiểm định Chi2 để nghiên cứu ảnh hưởng của Heart (lượt theo dõi) đến Star (Đánh giá)
#### Phát biểu giả thiết

$H_0: \mu_0 = \mu_1$. Heart không ảnh hưởng đến Star

$H_1: \mu_0 \neq \mu_1$. Heart ảnh hưởng đến Star

In [206]:
Type_heart = []
for i in df_2['heart']:
  if i <= 200:
    Type_heart.append('Low')
  elif 200 < i <= 500:
    Type_heart.append('Medium')
  elif 500 < i <= 1000:
    Type_heart.append('High')
  elif 1000 < i <= 2000:
    Type_heart.append('Super_High')
  else:
    Type_heart.append('Mega_High')
df_2['Type_heart'] = pd.Series(Type_heart)

In [207]:
Type_star = []
for i in df_2['star']:
  if i <= 4: Type_star.append('Low')
  elif 4 < i <= 4.5: Type_star.append('Medium')
  elif 4.5 < i <= 4.8: Type_star.append('Standard')
  else: Type_star.append('High')
df_2['Type_star'] = pd.Series(Type_star)
df_2.sample(5)

,artist,author,gernes,heart,name,star,view,words,Type_view,Type_words,Type_heart,Type_star
1017,Won,Nakanomura Ayasuke,"Action,Adventure,Comedy,Ecchi,Fantasy,Harem,Ma...",3717,Yuusha Densetsu no Uragawa de Ore wa Eiyuu Den...,4.9,2430032,255422,Mega_High,Super_High,Mega_High,High
1586,None,pilosopotasya,"Drama,Josei,Romance,Slice of Life,Web Novel,Co...",52,23/11,5.0,11639,37207,Low,Medium,Low,High
617,Yuzuki Kei:雪月佳,Kino Yuuki:木野裕喜,"Comedy,Ecchi,Fantasy,Gender Bender,Romance",167,"Because I Reincarnated as a Succubus, I’ll Squ...",4.8,41591,39065,Medium,Medium,Low,Standard
366,Seiji Kikuchi,Yoshioka Tsuyoshi,"Action,Comedy,Fantasy,Adventure,School Life,Ro...",560,Magi's Grandson,3.9,226296,163853,Super_High,Super_High,High,Low
888,None,Hiyoko no kēki,"Comedy,Romance,School Life,Slice of Life,Shoujo",570,"Kenkyo, Kenjitsu o Motto ni Ikite Orimasu",4.9,192873,86709,Super_High,High,High,High


In [208]:
Type_heart_star = df_2[['Type_heart', 'Type_star']].copy()
Type_heart_star.sample(5)

,Type_heart,Type_star
423,Medium,High
1101,Medium,High
743,Low,Low
920,Medium,Standard
336,Low,Medium


In [209]:
tanso_df = pd.crosstab(index=Type_heart_star['Type_heart'], columns=Type_heart_star['Type_star'])
tanso_df

Type_star,High,Low,Medium,Standard
Type_heart,,,,
High,182,5,19,119
Low,395,69,126,210
Medium,286,20,57,198
Mega_High,70,0,0,27
Super_High,106,1,2,58


In [210]:
score, p_value, dof, expected = st.chi2_contingency(tanso_df)

In [211]:
p ={}
p['score'] = score
p['p_value'] = p_value
p['dof'] = dof
results_1(p)

,score,p_value,dof,KetLuan
,121.712983,2.813703e-20,12,Chấp nhận H1 với mức ý nghĩa 0.05


#### 4. Dùng kiểm định Chi2 để nghiên cứu ảnh hưởng của Words (số chữ) đến Heart (lượt theo dõi)
#### Phát biểu giả thiết

$H_0: \mu_0 = \mu_1$. Words không ảnh hưởng đến Heart

$H_1: \mu_0 \neq \mu_1$. Words ảnh hưởng đến Heart

In [212]:
Type_words_heart = df_2[["Type_words", 'Type_heart']].copy()
Type_words_heart.head(5)

,Type_words,Type_heart
0,Low,Low
1,Mega_High,Mega_High
2,Medium,Low
3,Medium,Low
4,High,Super_High


In [213]:
tanso_df = pd.crosstab(index=Type_words_heart['Type_words'], columns=Type_words_heart['Type_heart'])
tanso_df

Type_heart,High,Low,Medium,Mega_High,Super_High
Type_words,,,,,
High,88,49,106,25,45
Low,47,528,177,1,7
Medium,121,209,235,6,52
Mega_High,16,4,20,41,28
Super_High,53,10,23,24,35


In [214]:
score, p_value, dof, expected = st.chi2_contingency(tanso_df)

In [215]:
p ={}
p['score'] = score
p['p_value'] = p_value
p['dof'] = dof
results_1(p)

,score,p_value,dof,KetLuan
,915.433304,1.395110e-184,16,Chấp nhận H1 với mức ý nghĩa 0.05


#### 5. Dùng kiểm định Anova
#### Phát biểu giả thiết

$H_0: \mu_1 = \mu_2 = \mu_3$. 

$H_1: \mu_1 \neq \mu_2 \neq \mu3$.

In [216]:
df_3 = df_2[['name', 'gernes', 'heart']]

In [217]:
a  = []
b = []
for i in range(len(df_3)):
  isA = False
  x = df_3['gernes'].iloc[i].split(',')
  for y in x:
    y = y.strip()
    if y == 'Romance':
      isA = True
  if isA == True:
    a.append(df_3['heart'].iloc[i])
    b.append(df_3['gernes'].iloc[i])
b = pd.Series(b, name = 'gernes')
a = pd.Series(a, name = "Heart")
c =pd.concat([b,a], axis = 1)

In [218]:
def Gernes(a,b,c):
  k = []
  l = []
  for i in range (len(df_3)):
    isA = False
    isB = False
    isC = False
    x = df_3['gernes'].iloc[i].split(',')
    for y in x:
      y = y.strip()
      if y == a:
        isA = True
      if y == b:
        isB = True
      if y == c:
        isC = True
    if isA == True and isB == True and isC == False:
      k.append(a + ' - ' + b)
      l.append(df_3['heart'].iloc[i])
  n = pd.Series(k, name = 'Category')
  m = pd.Series(l, name = 'Follows')
  t = pd.concat([n,m], axis = 1)
  return t

In [219]:
Type_1 = Gernes('Action', 'Comedy', 'Romance')
Type_2 = Gernes('Action', 'Romance', 'Comedy')
Type_3 = Gernes('Comedy', 'Romance', 'Action')

##### Thông kê đơn giản bằng hàm describle

In [220]:
type_1 = Type_1['Follows']
type_1.describe()

count     195.000000
mean      608.533333
std       967.280113
min        17.000000
25%       144.500000
50%       315.000000
75%       612.500000
max      8731.000000
Name: Follows, dtype: float64

In [221]:
type_2 = Type_2['Follows']
type_2.describe()

count     183.000000
mean      590.737705
std       631.088643
min        24.000000
25%       190.000000
50%       349.000000
75%       720.500000
max      2888.000000
Name: Follows, dtype: float64

In [222]:
type_3 = Type_3['Follows']
type_3.describe()

count     440.000000
mean      704.943182
std       810.114182
min         8.000000
25%       208.000000
50%       432.000000
75%       858.750000
max      6336.000000
Name: Follows, dtype: float64

### Tiến hành phân tích ANOVA 1 chiều

In [223]:
# Kiểm tra Type_1
from scipy.stats import shapiro
stat_type_1, p_type_1 = shapiro(type_1)
print('stat=%.3f, p=%.3f' % (stat_type_1, p_type_1))
if p_type_1 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

stat=0.539, p=0.000
Dữ liệu không tuân theo luật phân phối chuẩn


In [224]:
# Kiểm tra Type_2

stat_type_2, p_type_2 = shapiro(type_2)
print('stat=%.3f, p=%.3f' % (stat_type_2, p_type_2))
if p_type_2 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

stat=0.755, p=0.000
Dữ liệu không tuân theo luật phân phối chuẩn


In [225]:
# Kiểm tra Type_3

stat_type_3, p_type_3 = shapiro(type_3)
print('stat=%.3f, p=%.3f' % (stat_type_3, p_type_3))
if p_type_3 > 0.05:
    print('Không thể bác bỏ giả thiết H0: Phân phối của dữ liệu là chuẩn')
    print('Đủ điều kiện để phân tích tiếp')
else:
    print('Dữ liệu không tuân theo luật phân phối chuẩn')

stat=0.717, p=0.000
Dữ liệu không tuân theo luật phân phối chuẩn


In [226]:
# Vì data không tuân theo luật phân phối chuẩn nên ta sử dụng kiểm định kruskal-wallis 

from scipy.stats import kruskal

stats, p_value = kruskal(type_1,type_2,type_3)
print(stats)
print(p_value)
alpha = 0.05
if (p_value < alpha):
    print('Có bằng chứng bác bỏ giả thuyết H0. Điều này ngụ ý rằng số lượt theo dõi có sự khác biệt')
else:
    print('Không đủ bằng chứng bác bỏ giả thuyết H0. Điều này ngụ ý rằng số lượt theo dõi không có sự khác biệt.')

11.604944479491868
0.0030200791487424183
Có bằng chứng bác bỏ giả thuyết H0. Điều này ngụ ý rằng số lượt theo dõi có sự khác biệt
